In [2]:
import numpy as np
import tensorflow as tf

import plotly.express as px
import plotly.graph_objects as go

def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [3]:
max_wavevector_component_harmonic = 7
num_quasiwavevector_components = 99

In [4]:
wavevector_component_harmonics = np.arange(-max_wavevector_component_harmonic, max_wavevector_component_harmonic + 1)
wavevector_harmonics = cartesian_product(*(wavevector_component_harmonics,)*2)

position_components = np.linspace(-1/2, 1/2, wavevector_component_harmonics.size) # [spacing]

def wavevector_harmonic_to_index(wavevector_harmonic):
    return tuple(np.array(wavevector_harmonic) + np.full(2, max_wavevector_component_harmonic))

wavevector_harmonics.shape

(225, 2)

In [5]:
quasiwavevector_components = np.linspace(-1, 1, num_quasiwavevector_components) # [wavenumber]
quasiwavevectors = cartesian_product(*(quasiwavevector_components,)*2)

zero_quasiwavevector_index = (quasiwavevector_components.size - 1) // 2
max_quasiwavevector_index = -1

quasiwavevectors.shape

(9801, 2)

In [8]:
def compute_potential(depth, height):
    potential = np.zeros((wavevector_component_harmonics.size,)*2, dtype=complex)

    def add(indices, value):
        for harmonic in indices:
            potential[wavevector_harmonic_to_index(harmonic)] += value
    
    add((( 0,  0)),                               -1/4  * depth)
    add(((-2,  0), (+2,  0), ( 0, -2), ( 0, +2)), -1/8  * depth)
    add(((-2, -2), (-2, +2), (+2, -2), (+2, +2)), -1/16 * depth)

    add((( 0,  0)),                               +1/4  * height)
    add(((-1,  0), (+1,  0), ( 0, -1), ( 0, +1)), -1/8  * height)
    add(((-1, -1), (-1, +1), (+1, -1), (+1, +1)), +1/16 * height)

    return potential

In [23]:
def plot_potential(potential, periods=1):
    spatial_potential = -np.abs(np.fft.fftshift(np.fft.fft2(np.fft.fftshift(potential))))
    tiled_spatial_potential = np.tile(np.tile(spatial_potential, periods).transpose(), periods).transpose()
    position_components = np.linspace(-1/2 * periods, 1/2 * periods, wavevector_component_harmonics.size * periods)
    figure = go.Figure(data=[go.Surface(x=position_components, y=position_components, z=tiled_spatial_potential)])
    figure.update_layout(autosize=False, width=750, height=750)
    figure.show()

In [9]:
def compute_tunneling_strengths(energies):
    return np.abs(np.fft.fftshift(np.fft.fft2(np.fft.fftshift(energies[:, :, 0])))) / quasiwavevector_components.size**2

In [10]:
def compute_neighbor_tunneling_strengths(tunneling_strengths, radius=1):
    index_min = quasiwavevector_components.size // 2 - radius
    index_max = quasiwavevector_components.size // 2 + radius
    return tunneling_strengths[index_min:index_max+1, index_min:index_max+1]

In [11]:
effective_wavevectors = quasiwavevectors[:, np.newaxis, :] + 2*wavevector_harmonics
kinetic_energies = np.einsum('ijk, ijk -> ij', effective_wavevectors, effective_wavevectors).reshape(quasiwavevector_components.size, quasiwavevector_components.size, wavevector_harmonics.shape[0])

In [12]:
wavevector_change = wavevector_harmonics[:, np.newaxis] - wavevector_harmonics

In [13]:
RECOIL_ENERGY_752 = 1032 # Hz
RECOIL_ENERGY_532 = 2062 # Hz

# Lieb

In [61]:
potential = compute_potential(10, 10)

In [62]:
transition_energies = tf.cast(tf.linalg.diag(kinetic_energies), dtype=tf.complex128) + np.array([ [ potential[tuple(np.clip(wavevector_harmonic_to_index(y), 0, 14))] for y in x ] for x in wavevector_change ])[np.newaxis, np.newaxis, :]
transition_energies.shape

TensorShape([99, 99, 225, 225])

In [63]:
energies = tf.math.real(tf.linalg.eigvalsh(transition_energies))

In [64]:
height = np.max(energies[:, :, 0]) - np.min(energies[:, :, 0])
print(f"{height:.3f}")

0.932


In [1]:
figure = go.Figure(data=[go.Surface(x=quasiwavevector_components, 
                                    y=quasiwavevector_components, 
                                    z=energies[:, :, 1])])
figure.update_layout(autosize=False, width=750, height=750)
figure.show()

NameError: name 'go' is not defined

In [21]:
tunneling_strengths = compute_tunneling_strengths(energies)
neighbor_tunneling_strengths = compute_neighbor_tunneling_strengths(tunneling_strengths, radius=2)
px.imshow(neighbor_tunneling_strengths * RECOIL_ENERGY_752, zmax=neighbor_tunneling_strengths[2, 2+1] * RECOIL_ENERGY_752)